# Demonstration of the Metrics To-Date

For a complete list of metrics and their documentation, please see the API Metrics [documentation](../API/simulation_api.md#metrics-computation).

This demonstration will rely on the results produced in the "How To" notebook and serves as an extension of the API documentation to show what the results will look like depending on what inputs are provided.

<div class="alert alert-block alert-warning">
<b>Note:</b>
As of v0.5, most outputs, including availability, return values as a pandas DataFrame, regardless of the aggregation level.
</div>


In [1]:
from pprint import pprint

import pandas as pd

from wombat.core import Simulation, Metrics

# Clean up the aesthetics for the pandas outputs
pd.set_option("display.float_format", '{:,.2f}'.format)
pd.set_option("display.max_rows", 30)
pd.set_option("display.max_columns", 10)

## Table of Contents

Below is a list of top-level sections to demonstrate how to use WOMBAT's `Metrics` class methods and an explanation of each individual metric.
 - [Setup](#setup): Running a simulation to gather the results
 - [Availability](#availability): Time-based and energy-based availability
 - [Capacity Factor](#capacity-factor): Gross and net capacity factor
 - [Task Completion Rate](#task-completion-rate): Task completion metrics
 - [Equipment Costs](#equipment-costs): Cost breakdowns by servicing equipment
 - [Service Equipment Utilization Rate](#service-equipment-utilization-rate): Utilization of servicing equipment
 - [Vessel-Crew Hours at Sea](#vessel-crew-hours-at-sea): Number of crew or vessel hours spent at sea
 - [Number of Tows](#number-of-tows): Number of tows breakdowns
 - [Labor Costs](#labor-costs): Breakdown of labor costs
 - [Equipment and Labor Costs](#equipment-and-labor-costs): Combined servicing equipment and labor cost breakdown
 - [Component Costs](#component-costs): Materials costs
 - [Fixed Cost Impacts](#fixed-cost-impacts): Total fixed costs
 - [OpEx](#opex): Project OpEx
 - [Process Times](#process-times): Timing of various stages of repair and maintenance
 - [Power Production](#power-production): Potential and actually produced power
 - [Net Present Value](#net-present-value): Project NPV calculator
 - [PySAM-Powered Results](#pysam-powered-results): PySAM results, if configuration is provided

<a id="setup"></a>
## Setup

The simulations from the How To notebook are going to be rerun as it is not recommended to create a Metrics class from scratch due to the
large number of inputs that are required and the initialization is provided in the simulation API's run method.

To simplify this process, a feature has been added to save the simulation outputs required to generate the Metrics inputs and a method to reload those outputs as inputs.

In [2]:
sim = Simulation("DINWOODIE", "base.yaml")

# Both of these parameters are True by default for convenience
sim.run(create_metrics=True, save_metrics_inputs=True)

# Load the metrics data
fpath = sim.env.metrics_input_fname.parent
fname = sim.env.metrics_input_fname.name
metrics = Metrics.from_simulation_outputs(fpath, fname)

# Delete the log files now that they're loaded in
sim.env.cleanup_log_files(log_only=False)

# Alternatively, in this case because the simulation was run, we can use the
# following for convenience convenience only
metrics = sim.metrics

<a id="availability"></a>
## Availability

There are two methods to produce availability, which have their own function calls:
 - energy: `production_based_availability`
 - time: `time_based_availability`

Here, we will go through the various input definitions to get time-based availability data as both methods use the same inputs, and provide outputs in the same format.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [3]:
# Project total at the whole windfarm level
total = metrics.time_based_availability(frequency="project", by="windfarm").values[0][0]
print(f"Project total: {total * 100:.1f}%")

Project total: 94.2%


In [4]:
# Project total at the turbine level
metrics.time_based_availability(frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,...,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.81,0.89,0.94,0.86,0.93,...,0.97,0.93,0.97,0.97,0.97


In [5]:
# Project annual totals at the windfarm level
metrics.time_based_availability(frequency="annual", by="windfarm")

,windfarm
year,
2003,0.96
2004,0.94
2005,0.95
2006,0.95
2007,0.94
2008,0.91
2009,0.92
2010,0.95
2011,0.95


In [6]:
# Project monthly totals at the windfarm level
metrics.time_based_availability(frequency="monthly", by="windfarm")

,windfarm
month,
1,0.94
2,0.94
3,0.94
4,0.94
5,0.94
6,0.94
7,0.94
8,0.95
9,0.95


In [7]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.time_based_availability(frequency="month-year", by="windfarm")

windfarm
year month          
2003 1          0.99
     2          0.97
     3          0.95
     4          0.94
     5          0.96
...              ...
2012 8          0.97
     9          0.96
     10         0.97
     11         0.97
     12         0.96

[120 rows x 1 columns]

<a id="capacity-factor"></a>
## Capacity Factor

Here, we will go through the various input definitions to get capacity factor data. The inputs are very similar to that of the availability calculation.

`which` options:
 - net: net capcity factor, actual production
 - gross: gross capacity factor, potential production

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by` options:
 - windfarm: computed across all turbines
 - turbine: computed for each turbine

In [8]:
# Project total at the whole windfarm level
cf = metrics.capacity_factor(which="net", frequency="project", by="windfarm").values[0][0]
print(f"  Net Capacity Factor: {cf:.2f}%")

cf = metrics.capacity_factor(which="gross", frequency="project", by="windfarm").values[0][0]
print(f"Gross Capacity Factor: {cf:.2f}%")

  Net Capacity Factor: 0.45%
Gross Capacity Factor: 0.48%


In [9]:
# Project total at the turbine level
metrics.capacity_factor(which="net", frequency="project", by="turbine")

,S00T1,S00T2,S00T3,S00T4,S00T5,...,S00T76,S00T77,S00T78,S00T79,S00T80
0,0.38,0.42,0.44,0.40,0.44,...,0.47,0.44,0.47,0.46,0.46


In [10]:
# Project annual totals at the windfarm level
metrics.capacity_factor(which="net", frequency="annual", by="windfarm")

,windfarm
year,
2003,0.41
2004,0.44
2005,0.46
2006,0.45
2007,0.48
2008,0.47
2009,0.44
2010,0.40
2011,0.48


In [11]:
# Project monthly totals at the windfarm level
metrics.capacity_factor(which="net", frequency="monthly", by="windfarm")

,windfarm
month,
1,0.59
2,0.49
3,0.49
4,0.40
5,0.38
6,0.31
7,0.32
8,0.35
9,0.45


In [12]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.capacity_factor(which="net", frequency="month-year", by="windfarm")

windfarm
year month          
2003 1          0.63
     2          0.36
     3          0.36
     4          0.51
     5          0.34
...              ...
2012 8          0.31
     9          0.55
     10         0.52
     11         0.53
     12         0.60

[120 rows x 1 columns]

<a id="task-completion-rate"></a>
## Task Completion Rate

Here, we will go through the various input definitions to get the task completion rates. The inputs are very similar to that of the availability calculation.

`which` options:
 - scheduled: scheduled maintenance only (classified as maintenace tasks in inputs)
 - unscheduled: unscheduled maintenance only (classified as failure events in inputs)
 - both: 

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis

In [13]:
# Project total at the whole windfarm level
total = metrics.task_completion_rate(which="scheduled", frequency="project").values[0][0]
print(f"  Scheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="unscheduled", frequency="project").values[0][0]
print(f"Unscheduled Task Completion Rate: {total * 100:.0f}%")

total = metrics.task_completion_rate(which="both", frequency="project").values[0][0]
print(f"    Overall Task Completion Rate: {total * 100:.0f}%")

  Scheduled Task Completion Rate: 99%
Unscheduled Task Completion Rate: 100%
    Overall Task Completion Rate: 100%


In [14]:
# Project annual totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="annual")

,Completion Rate
year,
2003,1.00
2004,0.97
2005,1.02
2006,0.99
2007,0.99
2008,1.00
2009,1.00
2010,1.00
2011,0.99


In [15]:
# Project monthly totals at the windfarm level
metrics.task_completion_rate(which="both", frequency="monthly")

,Completion Rate
month,
1,0.75
2,0.86
3,0.99
4,1.08
5,1.15
6,1.13
7,1.07
8,1.07
9,1.04


In [16]:
# Project month-by-year totals at the windfarm level
# NOTE: This is limited to the first two years for cleanliness of the notebook
metrics.task_completion_rate(which="both", frequency="month-year")

Completion Rate
year month                 
2003 1                 0.89
     2                 1.06
     3                 0.92
     4                 1.00
     5                 1.10
...                     ...
2012 8                 1.06
     9                 1.05
     10                1.00
     11                0.98
     12                0.97

[120 rows x 1 columns]

<a id="equipment-costs"></a>
## Equipment Costs

Here, we will go through the various input definitions to get the equipment cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_equipment` options:
 - `True`: computed across all equipment used
 - `False`: computed for each piece of equipment

In [17]:
# Project total at the whole windfarm level
total = metrics.equipment_costs(frequency="project", by_equipment=False)
print(f"Project total: ${total.values[0][0] / metrics.project_capacity:,.2f}/MW")

Project total: $396,213.52/MW


In [18]:
# Project totals at the equipment level
metrics.equipment_costs(frequency="project", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,"6,088,778.65","6,086,500.00","6,087,411.46","1,842,722.81","74,985,831.60"


In [19]:
# Project annual totals at the windfarm level
metrics.equipment_costs(frequency="annual", by_equipment=False)

,equipment_cost
year,
2003,"10,125,502.88"
2004,"5,599,667.53"
2005,"11,227,310.96"
2006,"6,715,346.17"
2007,"11,911,006.30"
2008,"9,955,860.24"
2009,"14,008,162.38"
2010,"9,418,900.17"
2011,"10,199,301.60"


In [20]:
# Project monthly totals at the equipment level
metrics.equipment_costs(frequency="monthly", by_equipment=True)

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
month,,,,,
1,"496,216.15","500,882.81","497,546.88","93,812.50","6,212,500.00"
2,"457,151.04","456,750.00","459,429.69","156,606.29","4,995,388.89"
3,"512,622.40","508,229.17","510,562.50","218,663.06","4,739,062.50"
4,"512,585.94","512,276.04","512,276.04","424,333.33","6,457,918.40"
5,"530,486.98","528,682.29","529,794.27",0.00,"7,365,590.28"
6,"508,338.54","509,541.67","508,375.00","179,189.46","6,530,303.82"
7,"530,669.27","531,216.15","531,216.15","85,500.00","3,238,253.47"
8,"530,085.94","530,632.81","529,010.42","188,016.33","1,690,625.00"
9,"507,190.10","509,541.67","504,109.38","340,020.83","3,831,156.25"


In [21]:
# Project month-by-year totals at the equipment level
# NOTE: This is limited to the two years only
metrics.equipment_costs(frequency="month-year", by_equipment=True)

Crew Transfer Vessel 1  Crew Transfer Vessel 2  \
year month                                                   
2003 1                   51,333.33               53,666.67   
     2                   46,356.77               46,356.77   
     3                   54,250.00               53,010.42   
     4                   51,625.00               51,625.00   
     5                   54,250.00               54,250.00   
...                            ...                     ...   
2012 8                   53,885.42               53,885.42   
     9                   50,312.50               50,184.90   
     10                  50,458.33               53,138.02   
     11                  51,333.33               51,333.33   
     12                  52,117.19               51,041.67   

            Crew Transfer Vessel 3  Field Support Vessel  Heavy Lift Vessel  
year month                                                                   
2003 1                   53,010.42                  0.00               0.00  
     2                   47,250.00                  0.00               0.00  
     3                   54,250.00                  0.00               0.00  
     4                   51,625.00                  0.00               0.00  
     5                   53,447.92                  0.00       3,832,979.17  
...                            ...                   ...                ...  
2012 8                   53,885.42            150,199.29               0.00  
     9                   47,614.58            114,000.00               0.00  
     10                  51,187.50                  0.00               0.00  
     11                  52,500.00                  0.00               0.00  
     12                  53,101.56                  0.00               0.00  

[120 rows x 5 columns]

<a id="service-equipment-utilization-rate"></a>
## Service Equipment Utilization Rate

Here, we will go through the various input definitions to get the service equipment utiliztion rates.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis

In [22]:
# Project totals at the project level
total = metrics.service_equipment_utilization(frequency="project")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
0,1.00,1.00,1.00,0.93,0.95


In [23]:
# Annualized project totals
total = metrics.service_equipment_utilization(frequency="annual")
total

,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Field Support Vessel,Heavy Lift Vessel
year,,,,,
2003,1.00,1.00,1.00,0.95,0.94
2004,1.00,1.00,1.00,0.91,0.89
2005,1.00,1.00,1.00,0.93,0.96
2006,1.00,1.00,1.00,0.94,0.93
2007,1.00,1.00,1.00,0.93,0.95
2008,1.00,1.00,1.00,0.00,0.95
2009,1.00,1.00,1.00,0.94,0.96
2010,1.00,1.00,1.00,0.00,0.94
2011,1.00,1.00,1.00,0.94,0.96


<a id="vessel-crew-hours-at-sea"></a>
## Vessel-Crew Hours at Sea

The number of vessel hours or crew hours at sea for offshore wind power plant simulations.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_equipment` options:
 - `True`: computed for each vessel simulated
 - `False`: computed only as a sum of all vessels
 
`vessel_crew_assumption`: A dictionary of vessel names (`ServiceEquipment.settings.name`, but also found at `Metrics.service_equipment_names`) and the number of crew onboard at any given time. The application of this assumption transforms the results from vessel hours at sea to crew hours at sea.

In [24]:
# Project total, not broken out by vessel
metrics.vessel_crew_hours_at_sea(frequency="project", by_equipment=False)

,Total Crew Hours at Sea
0,"129,076.13"


In [25]:
# Annual project totals, broken out by vessel
metrics.vessel_crew_hours_at_sea(frequency="annual", by_equipment=True)

,year,Total Crew Hours at Sea,Crew Transfer Vessel 1,Crew Transfer Vessel 2,Crew Transfer Vessel 3,Heavy Lift Vessel,Field Support Vessel
0,2003,"8,844.76","1,469.75","1,528.50","1,663.75","3,557.76",625.00
1,2004,"12,025.97","3,420.75","3,340.75","3,404.25","1,828.22",32.00
2,2005,"14,153.02","3,269.75","3,269.50","3,274.00","3,739.77",600.00
3,2006,"12,596.36","3,284.00","3,382.75","3,317.75","2,008.86",603.00
4,2007,"14,268.06","3,398.25","3,341.50","3,314.75","3,613.56",600.00
5,2008,"13,265.01","3,267.75","3,284.50","3,223.50","3,489.26",0.00
6,2009,"15,303.78","3,269.00","2,922.00","3,141.50","5,341.78",629.50
7,2010,"13,414.16","3,252.75","3,336.00","3,361.25","3,464.16",0.00
8,2011,"12,737.75","3,441.75","3,249.25","3,391.25","2,052.50",603.00
9,2012,"12,467.25","3,293.00","3,417.00","3,395.00","1,762.25",600.00


<a id="number-of-tows"></a>
## Number of Tows

The number of tows metric will only produce results if any towing actually occurred, otherwise a dataframe with a value of 0 is returned, as is the case in this demonstration.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_tug` options:
 - `True`: computed for each tugboat (towing vessel)
 - `False`: computed only as a sum of all tugboats
 
`by_direction` options:
 - `True`: computed for each direction (to port or to site)
 - `False`: computed as a sum for both directions

In [26]:
# Project Total
# NOTE: This example has no towing, so it will return 0
metrics.number_of_tows(frequency="project")

,total_tows
0,0


<a id="labor-costs"></a>
## Labor Costs

Here, we will go through the various input definitions to get the labor cost data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_type` options:
 - `True`: computed across each labor type
 - `False`: computed for both labor types used

In [27]:
# Project total at the whole windfarm level
total = metrics.labor_costs(frequency="project", by_type=False)
print(f"Project total: ${total.values[0][0] / metrics.project_capacity:,.2f}/MW")

Project total: $0.00/MW


In [28]:
# Project totals for each type of labor
# NOTE: Only salaried labor was defined for thesese analyses
metrics.labor_costs(frequency="project", by_type=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost
0,0.00,0.00,0.00


In [29]:
# Project annual totals for all labor
metrics.labor_costs(frequency="annual", by_type=False)

,total_labor_cost
year,
2003,0.00
2004,0.00
2005,0.00
2006,0.00
2007,0.00
2008,0.00
2009,0.00
2010,0.00
2011,0.00


In [30]:
# Project monthly totals for all labor
metrics.labor_costs(frequency="monthly", by_type=False)

,total_labor_cost
month,
1,0.00
2,0.00
3,0.00
4,0.00
5,0.00
6,0.00
7,0.00
8,0.00
9,0.00


In [31]:
# Project month-by-year totals for all labor
# NOTE: This is limited to the first two years only
metrics.labor_costs(frequency="month-year", by_type=False)

total_labor_cost
year month                  
2003 1                  0.00
     2                  0.00
     3                  0.00
     4                  0.00
     5                  0.00
...                      ...
2012 8                  0.00
     9                  0.00
     10                 0.00
     11                 0.00
     12                 0.00

[120 rows x 1 columns]

<a id="equipment-and-labor-costs"></a>
## Equipment and Labor Costs

Here, we will go through the various input definitions to get the equipment and labor cost data broken out by expense categories.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across as equipment, plus each labor type, plus totals
 - `False`: computed as single total
 
**NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`reason` definitions:
 - Maintenance: routine maintenance
 - Repair: unscheduled maintenance, ranging from inspections to replacements
 - Weather Delay: Any delays caused by unsafe weather conditions
 - No Requests: Equipment and labor is active, but there are no repairs or maintenance tasks to be completed
 - Not in Shift: Any time outside of the operating hours of the windfarm

In [32]:
# Project totals
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=False)

,total_cost
reason,
Maintenance,"2,943,281.25"
Repair,"18,867,656.25"
Crew Transfer,"1,098,375.00"
Site Travel,0.00
Mobilization,"8,500,000.00"
Weather Delay,"35,893,361.11"
No Requests,"8,956,722.70"
Not in Shift,"18,831,848.20"


In [33]:
# Project totals by each category
metrics.equipment_labor_cost_breakdowns(frequency="project", by_category=True)

,hourly_labor_cost,salary_labor_cost,total_labor_cost,equipment_cost,total_cost
reason,,,,,
Maintenance,0.00,0.00,0.00,"2,943,281.25","2,943,281.25"
Repair,0.00,0.00,0.00,"18,867,656.25","18,867,656.25"
Crew Transfer,0.00,0.00,0.00,"1,098,375.00","1,098,375.00"
Site Travel,0.00,0.00,0.00,0.00,0.00
Mobilization,0.00,0.00,0.00,"8,500,000.00","8,500,000.00"
Weather Delay,0.00,0.00,0.00,"35,893,361.11","35,893,361.11"
No Requests,0.00,0.00,0.00,"8,956,722.70","8,956,722.70"
Not in Shift,0.00,0.00,0.00,"18,831,848.20","18,831,848.20"


In [34]:
# Project annual totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="annual", by_category=False)

total_cost
year reason                    
2003 Maintenance       4,375.00
     Repair        2,544,833.33
     Crew Transfer   110,562.50
     Site Travel           0.00
     Mobilization  1,000,000.00
...                         ...
2012 Site Travel           0.00
     Mobilization    500,000.00
     Weather Delay 1,727,286.46
     No Requests     603,257.81
     Not in Shift  1,390,793.04

[80 rows x 1 columns]

In [35]:
# Project monthly totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="monthly", by_category=False)

total_cost
month reason                    
1     Maintenance     196,783.85
      Repair          756,388.02
      Crew Transfer    61,203.12
      Site Travel           0.00
      Mobilization    500,000.00
...                          ...
12    Site Travel           0.00
      Mobilization    500,000.00
      Weather Delay 4,511,726.56
      No Requests     757,708.61
      Not in Shift  1,402,947.92

[96 rows x 1 columns]

In [36]:
# Project month-by-year totals
# NOTE: This is limited to the first two years
metrics.equipment_labor_cost_breakdowns(frequency="month-year", by_category=False)

total_cost
year month reason                   
2003 1     Maintenance          0.00
           Repair          21,802.08
           Crew Transfer    2,333.33
           Site Travel          0.00
           Mobilization         0.00
...                              ...
2012 12    Site Travel          0.00
           Mobilization         0.00
           Weather Delay    6,489.58
           No Requests     92,166.67
           Not in Shift    22,713.54

[960 rows x 1 columns]

<a id="component-costs"></a>
## Component Costs

Here, we will go through the various input definitions to get the component cost data broken out by various categories.

**NOTE**: It should be noted that the the component costs will not sum up to the whole project operations costs because of delays that are not associated with any repair or maintenance task, such as no requests needing to be processed.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_category` options:
 - `True`: computed across each cost category (includes total)
 - `False`: computed as single total
 
 `by_action` options:
 - `True`: computed by each of "repair", "maintenance", and "delay"
 - `False`: computed as single total
 
**NOTE:** For this breakdown the expense category (reason) is distributed across the rows in addition to time.


`action` definitions:
 - maintenance: routine maintenance
 - repair: unscheduled maintenance, ranging from inspections to replacements
 - delay: Any delays caused by unsafe weather conditions or not being able to finish a process within a single shift

In [37]:
# Project totals by component
metrics.component_costs(frequency="project", by_category=False, by_action=False)

,total_cost
component,
,"17,613,329.53"
turbine,"114,602,414.98"


In [38]:
# Project totals by each category and action type
metrics.component_costs(frequency="project", by_category=True, by_action=True)

materials_cost  total_labor_cost  equipment_cost  \
component action                                                          
          delay                  0.00              0.00    9,113,329.53   
          maintenance            0.00              0.00            0.00   
          repair                 0.00              0.00            0.00   
turbine   delay                  0.00              0.00   54,568,602.48   
          maintenance            0.00              0.00    2,943,281.25   
          repair                 0.00              0.00   18,867,656.25   

                         total_cost  
component action                     
          delay        9,113,329.53  
          maintenance          0.00  
          repair               0.00  
turbine   delay       54,568,602.48  
          maintenance  2,943,281.25  
          repair      18,867,656.25

In [39]:
# Project annual totals by category
# NOTE: This is limited to the first two years
metrics.component_costs(frequency="annual", by_category=True, by_action=False).head(28)

materials_cost  total_labor_cost  equipment_cost    total_cost
year component                                                                
2003                      0.00              0.00    3,146,138.29  3,146,138.29
     turbine      3,185,000.00              0.00    6,979,364.58 10,164,364.58
2004                      0.00              0.00      922,138.02    922,138.02
     turbine      2,787,500.00              0.00    4,677,529.51  7,465,029.51
2005                      0.00              0.00    1,578,645.16  1,578,645.16
     turbine      4,944,000.00              0.00    9,648,665.80 14,592,665.80
2006                      0.00              0.00    1,109,811.45  1,109,811.45
     turbine      3,094,500.00              0.00    5,605,534.72  8,700,034.72
2007                      0.00              0.00    2,042,686.86  2,042,686.86
     turbine      3,913,500.00              0.00    9,868,319.44 13,781,819.44
2008                      0.00              0.00    1,451,044.27  1,451,044.27
     turbine      3,636,500.00              0.00    8,504,815.97 12,141,315.97
2009                      0.00              0.00    2,286,622.40  2,286,622.40
     turbine      5,024,500.00              0.00   11,721,539.98 16,746,039.98
2010                      0.00              0.00    2,852,729.17  2,852,729.17
     turbine      3,598,500.00              0.00    6,566,171.01 10,164,671.01
2011                      0.00              0.00    1,095,871.92  1,095,871.92
     turbine      3,821,500.00              0.00    9,103,429.69 12,924,929.69
2012                      0.00              0.00    1,127,642.00  1,127,642.00
     turbine      3,119,000.00              0.00    4,802,544.27  7,921,544.27

In [40]:
# Project monthly totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="monthly", by_category=True, by_action=False).head(28)

materials_cost  total_labor_cost  equipment_cost  \
month component                                                     
1                          0.00              0.00      989,731.77   
      turbine      1,647,500.00              0.00    6,811,226.56   
2                          0.00              0.00    1,271,762.54   
      turbine      2,717,000.00              0.00    5,253,563.37   
3                          0.00              0.00      260,817.71   
      turbine      3,848,500.00              0.00    6,228,321.92   
4                          0.00              0.00    2,152,829.68   
      turbine      4,098,500.00              0.00    6,266,560.08   
5                          0.00              0.00    2,647,565.10   
      turbine      4,339,500.00              0.00    6,306,988.72   
6                          0.00              0.00    1,191,231.13   
      turbine      4,667,000.00              0.00    7,044,517.36   
7                          0.00              0.00      797,742.19   
      turbine      3,203,500.00              0.00    4,119,112.85   
8                          0.00              0.00      550,352.26   
      turbine      2,788,500.00              0.00    2,918,018.23   
9                          0.00              0.00    2,296,653.65   
      turbine      1,589,500.00              0.00    3,395,364.58   
10                         0.00              0.00    2,440,023.44   
      turbine      2,710,000.00              0.00   11,075,796.01   
11                         0.00              0.00    1,745,463.54   
      turbine      2,517,500.00              0.00   10,021,560.76   
12                         0.00              0.00    1,269,156.52   
      turbine      2,997,500.00              0.00    8,036,884.55   

                   total_cost  
month component                
1                  989,731.77  
      turbine    8,458,726.56  
2                1,271,762.54  
      turbine    7,970,563.37  
3                  260,817.71  
      turbine   10,076,821.92  
4                2,152,829.68  
      turbine   10,365,060.08  
5                2,647,565.10  
      turbine   10,646,488.72  
6                1,191,231.13  
      turbine   11,711,517.36  
7                  797,742.19  
      turbine    7,322,612.85  
8                  550,352.26  
      turbine    5,706,518.23  
9                2,296,653.65  
      turbine    4,984,864.58  
10               2,440,023.44  
      turbine   13,785,796.01  
11               1,745,463.54  
      turbine   12,539,060.76  
12               1,269,156.52  
      turbine   11,034,384.55

In [41]:
# Project month-by-year totals
# NOTE: This is limited to the first two months
metrics.component_costs(frequency="month-year", by_category=True, by_action=False).head(28)

materials_cost  total_labor_cost  equipment_cost  \
year month component                                                     
2003 1                          0.00              0.00      117,705.73   
           turbine         56,000.00              0.00       40,304.69   
     2                          0.00              0.00       93,023.44   
           turbine         16,000.00              0.00       46,940.10   
     3                          0.00              0.00      132,252.60   
           turbine         54,000.00              0.00       29,257.81   
     4                          0.00              0.00      120,804.69   
           turbine         53,000.00              0.00       34,070.31   
     5                          0.00              0.00      896,312.50   
           turbine      1,412,500.00              0.00    3,098,614.58   
     6                          0.00              0.00      569,656.25   
           turbine         39,500.00              0.00       34,872.40   
     7                          0.00              0.00      134,239.58   
           turbine         20,000.00              0.00       28,510.42   
     8                          0.00              0.00      115,263.02   
           turbine         85,000.00              0.00       42,473.96   
     9                          0.00              0.00      604,580.73   
           turbine        405,000.00              0.00    1,347,988.72   
     10                         0.00              0.00      100,989.58   
           turbine        728,000.00              0.00    2,075,448.78   
     11                         0.00              0.00      121,825.52   
           turbine         34,500.00              0.00       32,302.08   
     12                         0.00              0.00      139,484.65   
           turbine        281,500.00              0.00      168,580.73   
2004 1                          0.00              0.00       72,630.21   
           turbine        214,000.00              0.00      170,807.29   
     2                          0.00              0.00        2,533.85   
           turbine        173,000.00              0.00      128,825.52   

                       total_cost  
year month component               
2003 1                 117,705.73  
           turbine      96,304.69  
     2                  93,023.44  
           turbine      62,940.10  
     3                 132,252.60  
           turbine      83,257.81  
     4                 120,804.69  
           turbine      87,070.31  
     5                 896,312.50  
           turbine   4,511,114.58  
     6                 569,656.25  
           turbine      74,372.40  
     7                 134,239.58  
           turbine      48,510.42  
     8                 115,263.02  
           turbine     127,473.96  
     9                 604,580.73  
           turbine   1,752,988.72  
     10                100,989.58  
           turbine   2,803,448.78  
     11                121,825.52  
           turbine      66,802.08  
     12                139,484.65  
           turbine     450,080.73  
2004 1                  72,630.21  
           turbine     384,807.29  
     2                   2,533.85  
           turbine     301,825.52

<a id="fixed-cost-impacts"></a>
## Fixed Cost Impacts

Here, we will go through the various input definitions to get the fixed cost data

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`resolution` options:
 - high: computed across the lowest itemized cost levels
 - medium: computed across overarching cost levels
 - low: computed as single total

In [42]:
# The resolution hierarchy for fixed costs
pprint(metrics.fixed_costs.hierarchy)

{'operations': {'annual_leases_fees': ['submerge_land_lease_costs',
                                       'transmission_charges_rights'],
                'environmental_health_safety_monitoring': [],
                'insurance': ['brokers_fee',
                              'operations_all_risk',
                              'business_interruption',
                              'third_party_liability',
                              'storm_coverage'],
                'labor': [],
                'onshore_electrical_maintenance': [],
                'operating_facilities': [],
                'operations_management_administration': ['project_management_administration',
                                                         'marine_management',
                                                         'weather_forecasting',
                                                         'condition_monitoring']}}


In [43]:
# Project totals at the highest level
metrics.project_fixed_costs(frequency="project", resolution="low")

,operations
0,"16,013,230.75"


In [44]:
# Project totals at the medium level
metrics.project_fixed_costs(frequency="project", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,0.00,"16,013,230.75"


In [45]:
# Project totals at the lowest level
metrics.project_fixed_costs(frequency="project", resolution="high")

,project_management_administration,marine_management,weather_forecasting,condition_monitoring,brokers_fee,...,transmission_charges_rights,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
0,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,"16,013,230.75"


In [46]:
# Project annualized totals at the medium level
metrics.project_fixed_costs(frequency="annual", resolution="medium")

,operations_management_administration,insurance,annual_leases_fees,operating_facilities,environmental_health_safety_monitoring,onshore_electrical_maintenance,labor
year,,,,,,,
2003,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2004,0.00,0.00,0.00,0.00,0.00,0.00,"1,604,391.58"
2005,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2006,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2007,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2008,0.00,0.00,0.00,0.00,0.00,0.00,"1,604,391.58"
2009,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2010,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"
2011,0.00,0.00,0.00,0.00,0.00,0.00,"1,600,008.00"


In [47]:
# Project annualized totals at the low level by month
metrics.project_fixed_costs(frequency="annual", resolution="low")

,operations
year,
2003,"1,600,008.00"
2004,"1,604,391.58"
2005,"1,600,008.00"
2006,"1,600,008.00"
2007,"1,600,008.00"
2008,"1,604,391.58"
2009,"1,600,008.00"
2010,"1,600,008.00"
2011,"1,600,008.00"


In [48]:
# Project annualized totals at the low level by year and month
metrics.project_fixed_costs(frequency="month-year", resolution="low")

operations
year month            
2003 1      135,891.09
     2      122,740.34
     3      135,891.09
     4      131,507.51
     5      135,891.09
...                ...
2012 8      135,891.09
     9      131,507.51
     10     135,891.09
     11     131,507.51
     12     135,891.09

[120 rows x 1 columns]

<a id="opex"></a>
## OpEx

Here, we will go through the various input definitions to get the project OpEx

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis

In [49]:
metrics.opex("project")

,OpEx
0,"148,228,975.26"


In [50]:
metrics.opex("annual")

,OpEx
year,
2003,"14,910,510.88"
2004,"9,991,559.12"
2005,"17,771,318.96"
2006,"11,409,854.17"
2007,"17,424,514.30"
2008,"15,196,751.83"
2009,"20,632,670.38"
2010,"14,617,408.17"
2011,"15,620,809.60"


In [51]:
metrics.opex("monthly")

,OpEx
month,
1,"10,807,369.24"
2,"10,482,880.06"
3,"11,696,550.53"
4,"13,832,964.83"
5,"14,652,964.72"
6,"14,217,823.56"
7,"9,479,265.94"
8,"7,615,781.40"
9,"8,596,593.30"


In [52]:
metrics.opex("month-year")

OpEx
year month             
2003 1       349,901.51
     2       278,703.88
     3       351,401.51
     4       339,382.51
     5     5,543,318.17
...                 ...
2012 8       794,246.63
     9       538,119.49
     10      383,674.94
     11      354,174.17
     12      373,651.51

[120 rows x 1 columns]

In [53]:
port_fees = pd.DataFrame([], columns=["port_fees"], index=metrics.labor_costs(frequency="month-year", by_type=False).index)
port_fees.fillna(0)

port_fees
year month           
2003 1              0
     2              0
     3              0
     4              0
     5              0
...               ...
2012 8              0
     9              0
     10             0
     11             0
     12             0

[120 rows x 1 columns]

<a id="process-times"></a>
## Process Times

There are no inputs for the process timing as it is a slow calculation, so aggregation is left to the user for now. The results corresond to the number of hours required to complete any of the repair or maintenance activities.

In [54]:
# Project totals at the project level
total = metrics.process_times()
total

,time_to_completion,process_time,downtime,N
category,,,,
annual service,"1,091,642.60","93,833.50","97,800.50",681
major repair,"85,033.11","1,527.07","1,575.50",24
major replacement,"215,125.69","10,220.53","215,125.69",53
manual reset,"345,255.75","40,205.75","38,863.25",5578
medium repair,"15,482.65","9,933.00","10,464.75",202
minor repair,"104,921.81","34,247.00","35,603.00",2242


<a id="power-production"></a>
## Power Production

Here, we will go through the various input definitions to get the power production data.

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`by_turbine` options:
 - `True`: computed for each turbines
 - `False`: computed for the whole windfarm

In [55]:
# Project total at the whole windfarm level
total = metrics.power_production(frequency="project", by_turbine=False)
total

,windfarm
Project Energy Production (kWh),"9,442,005,889.50"


In [56]:
# Project totals at the turbine level
metrics.power_production(frequency="project", by_turbine=True)

,windfarm,S00T1,S00T2,S00T3,S00T4,...,S00T76,S00T77,S00T78,S00T79,S00T80
Project Energy Production (kWh),"9,442,005,889.50","101,052,897.50","109,806,549.00","116,042,852.00","105,944,237.50",...,"122,457,985.50","116,368,926.00","122,406,844.00","121,644,190.00","121,497,004.00"


In [57]:
# Project annual totals for the windfarm
metrics.power_production(frequency="annual", by_turbine=False)

,windfarm
year,
2003,"852,440,277.50"
2004,"921,782,125.00"
2005,"969,981,948.50"
2006,"936,659,950.50"
2007,"1,003,588,891.00"
2008,"1,000,683,786.50"
2009,"917,018,169.50"
2010,"846,170,093.00"
2011,"1,006,000,126.50"


In [58]:
# Project monthly totals for the windfarm
metrics.power_production(frequency="monthly", by_turbine=False)

,windfarm
month,
1,"1,051,843,301.00"
2,"799,463,558.50"
3,"876,713,983.50"
4,"692,208,129.00"
5,"673,704,686.00"
6,"543,886,229.00"
7,"573,457,352.00"
8,"630,391,595.50"
9,"783,298,223.00"


In [59]:
# Project month-by-year totals for the windfarm
# NOTE: This is limited to the first two years only
metrics.power_production(frequency="month-year", by_turbine=False)

windfarm
year month               
2003 1     112,048,886.00
     2      58,049,781.50
     3      65,121,285.00
     4      88,218,550.50
     5      60,116,346.50
...                   ...
2012 8      54,960,573.50
     9      94,294,751.50
     10     93,190,333.50
     11     91,215,330.50
     12    107,547,155.00

[120 rows x 1 columns]

<a id="net-present-value"></a>
## Net Present Value

Here, we will go through the various input definitions to get the project NPV

`frequency` options:
 - project: computed across the whole simulation
 - annual: computed on a yearly basis
 - monthly: computed across years on a monthly basis
 - month-year: computed on a month-by-year basis
 
`discount_rate`: The rate of return that could be earned on alternative investments, by default 0.025. 

`offtake_price`: Price of energy, per MWh, by default 80.

In [60]:
metrics.npv("project")

,NPV
0,"193,457,843.68"


In [61]:
metrics.opex("annual")

,OpEx
year,
2003,"14,910,510.88"
2004,"9,991,559.12"
2005,"17,771,318.96"
2006,"11,409,854.17"
2007,"17,424,514.30"
2008,"15,196,751.83"
2009,"20,632,670.38"
2010,"14,617,408.17"
2011,"15,620,809.60"


In [62]:
metrics.opex("monthly")

,OpEx
month,
1,"10,807,369.24"
2,"10,482,880.06"
3,"11,696,550.53"
4,"13,832,964.83"
5,"14,652,964.72"
6,"14,217,823.56"
7,"9,479,265.94"
8,"7,615,781.40"
9,"8,596,593.30"


In [63]:
metrics.opex("month-year")

OpEx
year month             
2003 1       349,901.51
     2       278,703.88
     3       351,401.51
     4       339,382.51
     5     5,543,318.17
...                 ...
2012 8       794,246.63
     9       538,119.49
     10      383,674.94
     11      354,174.17
     12      373,651.51

[120 rows x 1 columns]

<a id="pysam-powered-results"></a>
## PySAM-Powered Results

For a number of project financial metrics, the PySAM library is utilized.

<div class="alert alert-block alert-warning">
<b>NOTE:</b> If a "SAM_settings" file is not provided to the simulation, then the following metrics will not be able to be calculated and will raise a `NotImplementedError`.
</div>

With the above warning in mind, the appropriate simulation outputs are provided as inputs to PySAM upon initialization to ensure all values are aligned.

### Net Present Value (NPV)

In [64]:
try:
    npv = metrics.pysam_npv()
    print(f"NPV: ${npv:,.0f}")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_npv()' cannot be calculated!


### Real Levelized Cost of Energy (LCOE)

In [65]:
try:
    lcoe = metrics.pysam_lcoe_real()
    print(f"Real LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_real()' cannot be calculated!


### Nominal Levelized Cost of Energy (LCOE)

In [66]:
try:
    lcoe = metrics.pysam_lcoe_nominal()
    print(f"Nominal LCOE: ${lcoe:,.2f}/kW")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_lcoe_nominal()' cannot be calculated!


### After-tax Internal Return Rate (IRR)

In [67]:
try:
    npv = metrics.pysam_irr()
    print(f"IRR: {npv:,.1f}%")
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_irr()' cannot be calculated!


### One Data Frame to Rule Them All

For this demonstration we will manually load a PySAM settings file and trigger the setup for demonstration purposes, but it should be noted that this practice should be avoided.

In [68]:
try:
    metrics.pysam_all_outputs()
except NotImplementedError as e:
    print(e)

No SAM inputs were provided, and 'pysam_all_outputs()' cannot be calculated!
